# Week 16 - Day 6: PPO Trading

## Proximal Policy Optimization for Algorithmic Trading

**Date:** Day 6 of Week 16 - Reinforcement Learning

---

## Learning Objectives

1. **PPO Algorithm Basics** - Understand the theory behind Proximal Policy Optimization
2. **Clipped Objectives** - Learn how PPO stabilizes training with clipped surrogate objectives
3. **Full Trading Environment** - Build a complete gym-style trading environment
4. **Training and Evaluation** - Train PPO agent and evaluate trading performance

---

## Why PPO for Trading?

PPO is one of the most successful deep reinforcement learning algorithms, offering:
- **Stability**: Constrains policy updates to prevent catastrophic changes
- **Sample Efficiency**: Better data utilization than vanilla policy gradient
- **Simplicity**: Easier to implement and tune than TRPO
- **Robustness**: Works well across diverse environments including finance

## Part 1: Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

## Part 2: PPO Algorithm Basics

### 2.1 Policy Gradient Fundamentals

The standard policy gradient objective is:

$$L^{PG}(\theta) = \hat{\mathbb{E}}_t \left[ \log \pi_\theta(a_t | s_t) \hat{A}_t \right]$$

Where:
- $\pi_\theta$ is the policy network
- $\hat{A}_t$ is the advantage estimate at time $t$

### 2.2 The Problem with Vanilla Policy Gradient

- Large policy updates can destroy learned behavior
- Training is unstable and sensitive to hyperparameters
- No constraint on how much the policy can change

### 2.3 PPO Solution: Clipped Surrogate Objective

PPO introduces the probability ratio:

$$r_t(\theta) = \frac{\pi_\theta(a_t | s_t)}{\pi_{\theta_{old}}(a_t | s_t)}$$

And the clipped objective:

$$L^{CLIP}(\theta) = \hat{\mathbb{E}}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]$$

Where $\epsilon$ (typically 0.1-0.3) limits how much the policy can change.

In [ ]:
# Visualize the PPO clipping mechanism
def visualize_ppo_clipping(epsilon=0.2):
    """
    Visualize how PPO clips the objective function
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Ratio values
    r = np.linspace(0, 2, 1000)
    
    # Case 1: Positive advantage (A > 0)
    ax1 = axes[0]
    A_pos = 1.0
    
    # Unclipped objective
    L_unclipped_pos = r * A_pos
    
    # Clipped objective
    r_clipped = np.clip(r, 1 - epsilon, 1 + epsilon)
    L_clipped_pos = r_clipped * A_pos
    
    # PPO objective (min of both)
    L_ppo_pos = np.minimum(L_unclipped_pos, L_clipped_pos)
    
    ax1.plot(r, L_unclipped_pos, 'b--', label='Unclipped', alpha=0.7, linewidth=2)
    ax1.plot(r, L_clipped_pos, 'r--', label='Clipped', alpha=0.7, linewidth=2)
    ax1.plot(r, L_ppo_pos, 'g-', label='PPO (min)', linewidth=3)
    ax1.axvline(x=1, color='gray', linestyle=':', alpha=0.5)
    ax1.axvline(x=1-epsilon, color='orange', linestyle='--', alpha=0.5, label=f'Clip bounds')
    ax1.axvline(x=1+epsilon, color='orange', linestyle='--', alpha=0.5)
    ax1.fill_between(r, L_ppo_pos, alpha=0.2, color='green')
    ax1.set_xlabel('Probability Ratio r(θ)', fontsize=12)
    ax1.set_ylabel('Objective L', fontsize=12)
    ax1.set_title('Positive Advantage (A > 0)\nIncentivizes action, but clips to prevent too much change', fontsize=11)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 2)
    
    # Case 2: Negative advantage (A < 0)
    ax2 = axes[1]
    A_neg = -1.0
    
    L_unclipped_neg = r * A_neg
    L_clipped_neg = r_clipped * A_neg
    L_ppo_neg = np.minimum(L_unclipped_neg, L_clipped_neg)
    
    ax2.plot(r, L_unclipped_neg, 'b--', label='Unclipped', alpha=0.7, linewidth=2)
    ax2.plot(r, L_clipped_neg, 'r--', label='Clipped', alpha=0.7, linewidth=2)
    ax2.plot(r, L_ppo_neg, 'g-', label='PPO (min)', linewidth=3)
    ax2.axvline(x=1, color='gray', linestyle=':', alpha=0.5)
    ax2.axvline(x=1-epsilon, color='orange', linestyle='--', alpha=0.5, label=f'Clip bounds')
    ax2.axvline(x=1+epsilon, color='orange', linestyle='--', alpha=0.5)
    ax2.fill_between(r, L_ppo_neg, alpha=0.2, color='green')
    ax2.set_xlabel('Probability Ratio r(θ)', fontsize=12)
    ax2.set_ylabel('Objective L', fontsize=12)
    ax2.set_title('Negative Advantage (A < 0)\nDiscourages action, but clips to prevent too much change', fontsize=11)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, 2)
    
    plt.tight_layout()
    plt.suptitle(f'PPO Clipped Objective (ε = {epsilon})', fontsize=14, y=1.02)
    plt.show()
    
    print("\n📊 Key Insights:")
    print("• When A > 0: PPO encourages increasing action probability, but caps the benefit")
    print("• When A < 0: PPO encourages decreasing action probability, but limits the penalty")
    print(f"• The clipping range [{1-epsilon:.1f}, {1+epsilon:.1f}] creates a 'trust region'")

visualize_ppo_clipping(epsilon=0.2)

## Part 3: Full Trading Environment

### 3.1 Environment Design

Our trading environment will have:
- **State Space**: Price features, technical indicators, portfolio state
- **Action Space**: Hold (0), Buy (1), Sell (2)
- **Reward**: Portfolio return with risk adjustment

In [ ]:
# Download market data
def download_data(ticker='SPY', start='2018-01-01', end='2024-01-01'):
    """
    Download and prepare market data
    """
    print(f"📥 Downloading {ticker} data...")
    df = yf.download(ticker, start=start, end=end, progress=False)
    
    # Use only Close price as requested
    if isinstance(df.columns, pd.MultiIndex):
        df = df['Close'].to_frame(name='Close')
    else:
        df = df[['Close']]
    
    df = df.dropna()
    print(f"✅ Downloaded {len(df)} trading days")
    print(f"   Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
    
    return df

# Download data
df = download_data('SPY', start='2018-01-01', end='2024-01-01')
df.head()

In [ ]:
class TradingEnvironment:
    """
    A full trading environment for PPO
    
    Features:
    - Realistic transaction costs
    - Position tracking
    - Multiple technical indicators as state
    - Risk-adjusted rewards
    """
    
    def __init__(self, df, initial_balance=100000, transaction_cost=0.001, 
                 window_size=20, max_position=1.0):
        """
        Initialize trading environment
        
        Args:
            df: DataFrame with 'Close' prices
            initial_balance: Starting capital
            transaction_cost: Cost per trade (0.1%)
            window_size: Lookback window for features
            max_position: Maximum position size (1.0 = 100% of capital)
        """
        self.df = df.copy()
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost
        self.window_size = window_size
        self.max_position = max_position
        
        # Prepare features
        self._prepare_features()
        
        # Action space: 0=Hold, 1=Buy, 2=Sell
        self.action_space = 3
        self.state_dim = self._get_state_dim()
        
        self.reset()
    
    def _prepare_features(self):
        """
        Calculate technical indicators from Close price
        """
        close = self.df['Close']
        
        # Returns
        self.df['Returns'] = close.pct_change()
        
        # Moving averages
        self.df['SMA_10'] = close.rolling(10).mean()
        self.df['SMA_20'] = close.rolling(20).mean()
        self.df['SMA_50'] = close.rolling(50).mean()
        
        # Price relative to MAs
        self.df['Price_SMA10_Ratio'] = close / self.df['SMA_10']
        self.df['Price_SMA20_Ratio'] = close / self.df['SMA_20']
        self.df['SMA10_SMA20_Ratio'] = self.df['SMA_10'] / self.df['SMA_20']
        
        # Volatility
        self.df['Volatility_10'] = self.df['Returns'].rolling(10).std()
        self.df['Volatility_20'] = self.df['Returns'].rolling(20).std()
        
        # Momentum
        self.df['Momentum_5'] = close.pct_change(5)
        self.df['Momentum_10'] = close.pct_change(10)
        self.df['Momentum_20'] = close.pct_change(20)
        
        # RSI
        delta = close.diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        self.df['RSI'] = 100 - (100 / (1 + rs))
        self.df['RSI_Normalized'] = (self.df['RSI'] - 50) / 50  # Normalize to [-1, 1]
        
        # Bollinger Bands
        self.df['BB_Middle'] = close.rolling(20).mean()
        bb_std = close.rolling(20).std()
        self.df['BB_Upper'] = self.df['BB_Middle'] + 2 * bb_std
        self.df['BB_Lower'] = self.df['BB_Middle'] - 2 * bb_std
        self.df['BB_Position'] = (close - self.df['BB_Lower']) / (self.df['BB_Upper'] - self.df['BB_Lower'])
        
        # Drop NaN rows
        self.df = self.df.dropna()
        
        # Feature columns for state
        self.feature_columns = [
            'Returns', 'Price_SMA10_Ratio', 'Price_SMA20_Ratio', 'SMA10_SMA20_Ratio',
            'Volatility_10', 'Volatility_20', 'Momentum_5', 'Momentum_10', 'Momentum_20',
            'RSI_Normalized', 'BB_Position'
        ]
    
    def _get_state_dim(self):
        """
        Get state dimension: features + portfolio state
        """
        # Features + position + unrealized_pnl_pct + cash_ratio
        return len(self.feature_columns) + 3
    
    def reset(self):
        """
        Reset environment to initial state
        """
        self.current_step = 0
        self.balance = self.initial_balance
        self.shares_held = 0
        self.cost_basis = 0
        self.total_trades = 0
        self.total_profit = 0
        
        # Track history for analysis
        self.history = {
            'portfolio_value': [self.initial_balance],
            'actions': [],
            'positions': [0],
            'returns': []
        }
        
        return self._get_state()
    
    def _get_state(self):
        """
        Get current state observation
        """
        # Market features
        features = self.df[self.feature_columns].iloc[self.current_step].values
        
        # Portfolio state
        current_price = self.df['Close'].iloc[self.current_step]
        portfolio_value = self.balance + self.shares_held * current_price
        
        # Position as fraction of portfolio
        position = (self.shares_held * current_price) / portfolio_value if portfolio_value > 0 else 0
        
        # Unrealized P&L percentage
        if self.shares_held > 0 and self.cost_basis > 0:
            unrealized_pnl_pct = (current_price - self.cost_basis) / self.cost_basis
        else:
            unrealized_pnl_pct = 0
        
        # Cash ratio
        cash_ratio = self.balance / portfolio_value if portfolio_value > 0 else 1
        
        # Combine all state components
        state = np.concatenate([
            features,
            [position, unrealized_pnl_pct, cash_ratio]
        ])
        
        return state.astype(np.float32)
    
    def step(self, action):
        """
        Execute action and return next state, reward, done
        
        Args:
            action: 0=Hold, 1=Buy, 2=Sell
        
        Returns:
            next_state, reward, done, info
        """
        current_price = self.df['Close'].iloc[self.current_step]
        prev_portfolio_value = self.balance + self.shares_held * current_price
        
        # Execute action
        if action == 1:  # Buy
            self._buy(current_price)
        elif action == 2:  # Sell
            self._sell(current_price)
        # action == 0 is Hold, do nothing
        
        # Move to next step
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        
        # Get new price and portfolio value
        new_price = self.df['Close'].iloc[self.current_step]
        new_portfolio_value = self.balance + self.shares_held * new_price
        
        # Calculate reward (log return with risk adjustment)
        portfolio_return = (new_portfolio_value - prev_portfolio_value) / prev_portfolio_value
        reward = self._calculate_reward(portfolio_return, action)
        
        # Update history
        self.history['portfolio_value'].append(new_portfolio_value)
        self.history['actions'].append(action)
        self.history['positions'].append(self.shares_held * new_price / new_portfolio_value if new_portfolio_value > 0 else 0)
        self.history['returns'].append(portfolio_return)
        
        next_state = self._get_state()
        
        info = {
            'portfolio_value': new_portfolio_value,
            'position': self.shares_held,
            'balance': self.balance,
            'total_trades': self.total_trades
        }
        
        return next_state, reward, done, info
    
    def _buy(self, price):
        """
        Execute buy order
        """
        if self.balance > 0:
            # Calculate max shares we can buy
            max_investment = self.balance * self.max_position
            transaction_fee = max_investment * self.transaction_cost
            shares_to_buy = int((max_investment - transaction_fee) / price)
            
            if shares_to_buy > 0:
                cost = shares_to_buy * price
                fee = cost * self.transaction_cost
                
                self.balance -= (cost + fee)
                self.shares_held += shares_to_buy
                self.cost_basis = price
                self.total_trades += 1
    
    def _sell(self, price):
        """
        Execute sell order
        """
        if self.shares_held > 0:
            revenue = self.shares_held * price
            fee = revenue * self.transaction_cost
            
            profit = revenue - (self.shares_held * self.cost_basis) - fee
            self.total_profit += profit
            
            self.balance += (revenue - fee)
            self.shares_held = 0
            self.cost_basis = 0
            self.total_trades += 1
    
    def _calculate_reward(self, portfolio_return, action):
        """
        Calculate risk-adjusted reward
        """
        # Base reward is the portfolio return
        reward = portfolio_return * 100  # Scale for better learning
        
        # Penalty for excessive trading (transaction costs already applied)
        if action != 0:  # If not holding
            reward -= 0.01  # Small penalty for trading
        
        return reward
    
    def get_metrics(self):
        """
        Calculate performance metrics
        """
        portfolio_values = np.array(self.history['portfolio_value'])
        returns = np.array(self.history['returns']) if self.history['returns'] else np.array([0])
        
        # Total return
        total_return = (portfolio_values[-1] - self.initial_balance) / self.initial_balance
        
        # Annualized return
        n_days = len(portfolio_values)
        annualized_return = (1 + total_return) ** (252 / n_days) - 1 if n_days > 0 else 0
        
        # Sharpe ratio
        if len(returns) > 1 and np.std(returns) > 0:
            sharpe = np.sqrt(252) * np.mean(returns) / np.std(returns)
        else:
            sharpe = 0
        
        # Maximum drawdown
        peak = np.maximum.accumulate(portfolio_values)
        drawdown = (peak - portfolio_values) / peak
        max_drawdown = np.max(drawdown)
        
        # Win rate
        if len(returns) > 0:
            win_rate = np.sum(returns > 0) / len(returns)
        else:
            win_rate = 0
        
        return {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'win_rate': win_rate,
            'total_trades': self.total_trades,
            'final_value': portfolio_values[-1]
        }

print("✅ Trading Environment class defined")

In [ ]:
# Test the environment
env = TradingEnvironment(df, initial_balance=100000)
print(f"\n📊 Environment Configuration:")
print(f"   State dimension: {env.state_dim}")
print(f"   Action space: {env.action_space} (Hold, Buy, Sell)")
print(f"   Data points: {len(env.df)}")
print(f"   Feature columns: {env.feature_columns}")

# Test a few random steps
state = env.reset()
print(f"\n🔍 Initial state shape: {state.shape}")
print(f"   Initial state (first 5 values): {state[:5]}")

# Take some random actions
for i in range(5):
    action = np.random.randint(0, 3)
    next_state, reward, done, info = env.step(action)
    action_names = ['Hold', 'Buy', 'Sell']
    print(f"   Step {i+1}: Action={action_names[action]}, Reward={reward:.4f}, Portfolio=${info['portfolio_value']:,.2f}")

## Part 4: PPO Neural Network Architecture

### 4.1 Actor-Critic Network

PPO uses an Actor-Critic architecture:
- **Actor (Policy Network)**: Outputs action probabilities
- **Critic (Value Network)**: Estimates state value for advantage calculation

We'll use a shared backbone with separate heads for efficiency.

In [ ]:
class ActorCritic(nn.Module):
    """
    Actor-Critic network for PPO
    
    Uses shared layers with separate heads for policy and value
    """
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(ActorCritic, self).__init__()
        
        # Shared feature extractor
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU()
        )
        
        # Actor head (policy)
        self.actor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, action_dim),
            nn.Softmax(dim=-1)
        )
        
        # Critic head (value function)
        self.critic = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """
        Initialize network weights using orthogonal initialization
        """
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.orthogonal_(module.weight, gain=np.sqrt(2))
                nn.init.constant_(module.bias, 0)
    
    def forward(self, state):
        """
        Forward pass
        
        Returns:
            action_probs: Probability distribution over actions
            value: Estimated state value
        """
        shared_features = self.shared(state)
        action_probs = self.actor(shared_features)
        value = self.critic(shared_features)
        return action_probs, value
    
    def act(self, state):
        """
        Select action based on current policy
        
        Returns:
            action: Selected action
            log_prob: Log probability of selected action
            value: Estimated state value
        """
        action_probs, value = self.forward(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        return action.item(), log_prob, value
    
    def evaluate(self, states, actions):
        """
        Evaluate actions for PPO update
        
        Returns:
            log_probs: Log probabilities of actions
            values: State values
            entropy: Policy entropy
        """
        action_probs, values = self.forward(states)
        dist = Categorical(action_probs)
        
        log_probs = dist.log_prob(actions)
        entropy = dist.entropy()
        
        return log_probs, values.squeeze(-1), entropy

# Test the network
test_model = ActorCritic(state_dim=env.state_dim, action_dim=env.action_space)
test_state = torch.FloatTensor(env.reset()).unsqueeze(0)
action_probs, value = test_model(test_state)

print("✅ Actor-Critic Network")
print(f"   Input state dim: {env.state_dim}")
print(f"   Action probabilities: {action_probs.detach().numpy()[0]}")
print(f"   State value: {value.item():.4f}")
print(f"   Total parameters: {sum(p.numel() for p in test_model.parameters()):,}")

## Part 5: PPO Agent Implementation

### 5.1 Memory Buffer

PPO needs to store trajectories for batch updates.

In [ ]:
class RolloutBuffer:
    """
    Buffer to store rollout data for PPO updates
    """
    
    def __init__(self):
        self.clear()
    
    def clear(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.log_probs = []
        self.values = []
        self.dones = []
    
    def add(self, state, action, reward, log_prob, value, done):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.log_probs.append(log_prob)
        self.values.append(value)
        self.dones.append(done)
    
    def get(self):
        return (
            np.array(self.states),
            np.array(self.actions),
            np.array(self.rewards),
            np.array([lp.item() for lp in self.log_probs]),
            np.array([v.item() for v in self.values]),
            np.array(self.dones)
        )
    
    def __len__(self):
        return len(self.states)

print("✅ RolloutBuffer class defined")

In [ ]:
class PPOAgent:
    """
    Proximal Policy Optimization Agent for Trading
    
    Key features:
    - Clipped surrogate objective
    - Generalized Advantage Estimation (GAE)
    - Entropy bonus for exploration
    - Value function clipping
    """
    
    def __init__(self, state_dim, action_dim, 
                 lr=3e-4,
                 gamma=0.99,
                 gae_lambda=0.95,
                 clip_epsilon=0.2,
                 c1=0.5,  # Value loss coefficient
                 c2=0.01,  # Entropy coefficient
                 n_epochs=10,
                 batch_size=64,
                 hidden_dim=128):
        """
        Initialize PPO Agent
        
        Args:
            state_dim: Dimension of state space
            action_dim: Number of possible actions
            lr: Learning rate
            gamma: Discount factor
            gae_lambda: GAE parameter
            clip_epsilon: PPO clipping parameter
            c1: Value loss coefficient
            c2: Entropy bonus coefficient
            n_epochs: Number of epochs per update
            batch_size: Mini-batch size for updates
            hidden_dim: Hidden layer dimension
        """
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.clip_epsilon = clip_epsilon
        self.c1 = c1
        self.c2 = c2
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        
        # Actor-Critic network
        self.policy = ActorCritic(state_dim, action_dim, hidden_dim).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr, eps=1e-5)
        
        # Memory buffer
        self.buffer = RolloutBuffer()
        
        # Training statistics
        self.training_stats = {
            'policy_loss': [],
            'value_loss': [],
            'entropy': [],
            'total_loss': [],
            'clip_fraction': []
        }
    
    def select_action(self, state):
        """
        Select action using current policy
        """
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        
        with torch.no_grad():
            action, log_prob, value = self.policy.act(state_tensor)
        
        return action, log_prob, value
    
    def store_transition(self, state, action, reward, log_prob, value, done):
        """
        Store transition in buffer
        """
        self.buffer.add(state, action, reward, log_prob, value, done)
    
    def compute_gae(self, rewards, values, dones, last_value):
        """
        Compute Generalized Advantage Estimation
        
        GAE formula:
        A_t = δ_t + (γλ)δ_{t+1} + (γλ)²δ_{t+2} + ...
        where δ_t = r_t + γV(s_{t+1}) - V(s_t)
        """
        advantages = np.zeros_like(rewards)
        returns = np.zeros_like(rewards)
        
        gae = 0
        next_value = last_value
        
        for t in reversed(range(len(rewards))):
            if dones[t]:
                delta = rewards[t] - values[t]
                gae = delta
            else:
                delta = rewards[t] + self.gamma * next_value - values[t]
                gae = delta + self.gamma * self.gae_lambda * gae
            
            advantages[t] = gae
            returns[t] = advantages[t] + values[t]
            next_value = values[t]
        
        return advantages, returns
    
    def update(self):
        """
        Update policy using PPO algorithm
        """
        # Get data from buffer
        states, actions, rewards, old_log_probs, values, dones = self.buffer.get()
        
        # Get last value for GAE computation
        with torch.no_grad():
            last_state = torch.FloatTensor(states[-1]).unsqueeze(0).to(device)
            _, last_value = self.policy(last_state)
            last_value = last_value.item()
        
        # Compute advantages and returns
        advantages, returns = self.compute_gae(rewards, values, dones, last_value)
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # Convert to tensors
        states_tensor = torch.FloatTensor(states).to(device)
        actions_tensor = torch.LongTensor(actions).to(device)
        old_log_probs_tensor = torch.FloatTensor(old_log_probs).to(device)
        advantages_tensor = torch.FloatTensor(advantages).to(device)
        returns_tensor = torch.FloatTensor(returns).to(device)
        
        # PPO update epochs
        dataset_size = len(states)
        
        total_policy_loss = 0
        total_value_loss = 0
        total_entropy = 0
        total_clip_fraction = 0
        n_updates = 0
        
        for epoch in range(self.n_epochs):
            # Create random indices for mini-batches
            indices = np.random.permutation(dataset_size)
            
            for start in range(0, dataset_size, self.batch_size):
                end = min(start + self.batch_size, dataset_size)
                batch_indices = indices[start:end]
                
                # Get batch data
                batch_states = states_tensor[batch_indices]
                batch_actions = actions_tensor[batch_indices]
                batch_old_log_probs = old_log_probs_tensor[batch_indices]
                batch_advantages = advantages_tensor[batch_indices]
                batch_returns = returns_tensor[batch_indices]
                
                # Evaluate current policy
                new_log_probs, new_values, entropy = self.policy.evaluate(
                    batch_states, batch_actions
                )
                
                # Compute probability ratio
                ratio = torch.exp(new_log_probs - batch_old_log_probs)
                
                # Clipped surrogate objective
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * batch_advantages
                policy_loss = -torch.min(surr1, surr2).mean()
                
                # Value loss (MSE)
                value_loss = F.mse_loss(new_values, batch_returns)
                
                # Entropy bonus
                entropy_loss = -entropy.mean()
                
                # Total loss
                loss = policy_loss + self.c1 * value_loss + self.c2 * entropy_loss
                
                # Optimize
                self.optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(self.policy.parameters(), 0.5)
                self.optimizer.step()
                
                # Track statistics
                clip_fraction = (torch.abs(ratio - 1) > self.clip_epsilon).float().mean().item()
                total_policy_loss += policy_loss.item()
                total_value_loss += value_loss.item()
                total_entropy += entropy.mean().item()
                total_clip_fraction += clip_fraction
                n_updates += 1
        
        # Store average statistics
        self.training_stats['policy_loss'].append(total_policy_loss / n_updates)
        self.training_stats['value_loss'].append(total_value_loss / n_updates)
        self.training_stats['entropy'].append(total_entropy / n_updates)
        self.training_stats['clip_fraction'].append(total_clip_fraction / n_updates)
        
        # Clear buffer
        self.buffer.clear()
        
        return {
            'policy_loss': total_policy_loss / n_updates,
            'value_loss': total_value_loss / n_updates,
            'entropy': total_entropy / n_updates,
            'clip_fraction': total_clip_fraction / n_updates
        }

print("✅ PPO Agent class defined")

## Part 6: Training the PPO Agent

### 6.1 Training Loop

We'll train the agent over multiple episodes, collecting trajectories and updating the policy.

In [ ]:
def train_ppo(env, agent, n_episodes=100, update_freq=2048, verbose=True):
    """
    Train PPO agent on trading environment
    
    Args:
        env: Trading environment
        agent: PPO agent
        n_episodes: Number of training episodes
        update_freq: Steps between policy updates
        verbose: Print training progress
    
    Returns:
        training_history: Dict of training metrics
    """
    training_history = {
        'episode_rewards': [],
        'episode_returns': [],
        'sharpe_ratios': [],
        'max_drawdowns': [],
        'total_trades': []
    }
    
    best_sharpe = -np.inf
    total_steps = 0
    
    for episode in range(n_episodes):
        state = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            # Select action
            action, log_prob, value = agent.select_action(state)
            
            # Take step
            next_state, reward, done, info = env.step(action)
            
            # Store transition
            agent.store_transition(state, action, reward, log_prob, value, done)
            
            state = next_state
            episode_reward += reward
            total_steps += 1
            
            # Update policy
            if total_steps % update_freq == 0:
                update_stats = agent.update()
        
        # End of episode - update if buffer not empty
        if len(agent.buffer) > 0:
            agent.update()
        
        # Get episode metrics
        metrics = env.get_metrics()
        training_history['episode_rewards'].append(episode_reward)
        training_history['episode_returns'].append(metrics['total_return'])
        training_history['sharpe_ratios'].append(metrics['sharpe_ratio'])
        training_history['max_drawdowns'].append(metrics['max_drawdown'])
        training_history['total_trades'].append(metrics['total_trades'])
        
        # Track best model
        if metrics['sharpe_ratio'] > best_sharpe:
            best_sharpe = metrics['sharpe_ratio']
        
        # Print progress
        if verbose and (episode + 1) % 10 == 0:
            avg_reward = np.mean(training_history['episode_rewards'][-10:])
            avg_return = np.mean(training_history['episode_returns'][-10:])
            avg_sharpe = np.mean(training_history['sharpe_ratios'][-10:])
            
            print(f"Episode {episode + 1:3d}/{n_episodes} | "
                  f"Avg Reward: {avg_reward:8.2f} | "
                  f"Avg Return: {avg_return*100:6.2f}% | "
                  f"Avg Sharpe: {avg_sharpe:.3f} | "
                  f"Best Sharpe: {best_sharpe:.3f}")
    
    return training_history

print("✅ Training function defined")

In [ ]:
# Split data into train and test
train_size = int(len(df) * 0.7)
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()

print(f"📊 Data Split:")
print(f"   Training: {len(train_df)} days ({train_df.index[0].strftime('%Y-%m-%d')} to {train_df.index[-1].strftime('%Y-%m-%d')})")
print(f"   Testing:  {len(test_df)} days ({test_df.index[0].strftime('%Y-%m-%d')} to {test_df.index[-1].strftime('%Y-%m-%d')})")

# Create training environment
train_env = TradingEnvironment(train_df, initial_balance=100000)

# Initialize PPO agent
agent = PPOAgent(
    state_dim=train_env.state_dim,
    action_dim=train_env.action_space,
    lr=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_epsilon=0.2,
    c1=0.5,
    c2=0.01,
    n_epochs=10,
    batch_size=64,
    hidden_dim=128
)

print(f"\n🤖 PPO Agent Configuration:")
print(f"   Learning rate: 3e-4")
print(f"   Discount factor (γ): 0.99")
print(f"   GAE λ: 0.95")
print(f"   Clip ε: 0.2")
print(f"   Update epochs: 10")
print(f"   Batch size: 64")

In [ ]:
# Train the agent
print("\n🚀 Starting PPO Training...\n")
print("="*80)

training_history = train_ppo(
    env=train_env,
    agent=agent,
    n_episodes=50,  # Reduced for demo, increase for better results
    update_freq=1024,
    verbose=True
)

print("="*80)
print("\n✅ Training Complete!")

In [ ]:
# Plot training progress
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Episode rewards
ax1 = axes[0, 0]
ax1.plot(training_history['episode_rewards'], alpha=0.6, label='Episode Reward')
window = 10
if len(training_history['episode_rewards']) >= window:
    rolling_reward = pd.Series(training_history['episode_rewards']).rolling(window).mean()
    ax1.plot(rolling_reward, linewidth=2, label=f'{window}-Episode MA')
ax1.set_xlabel('Episode')
ax1.set_ylabel('Total Reward')
ax1.set_title('Training Rewards')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Episode returns
ax2 = axes[0, 1]
returns_pct = [r * 100 for r in training_history['episode_returns']]
ax2.plot(returns_pct, alpha=0.6, label='Episode Return')
if len(returns_pct) >= window:
    rolling_return = pd.Series(returns_pct).rolling(window).mean()
    ax2.plot(rolling_return, linewidth=2, label=f'{window}-Episode MA')
ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax2.set_xlabel('Episode')
ax2.set_ylabel('Return (%)')
ax2.set_title('Portfolio Returns')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Sharpe ratios
ax3 = axes[1, 0]
ax3.plot(training_history['sharpe_ratios'], alpha=0.6, label='Sharpe Ratio')
if len(training_history['sharpe_ratios']) >= window:
    rolling_sharpe = pd.Series(training_history['sharpe_ratios']).rolling(window).mean()
    ax3.plot(rolling_sharpe, linewidth=2, label=f'{window}-Episode MA')
ax3.axhline(y=0, color='r', linestyle='--', alpha=0.5)
ax3.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='Sharpe = 1')
ax3.set_xlabel('Episode')
ax3.set_ylabel('Sharpe Ratio')
ax3.set_title('Risk-Adjusted Performance')
ax3.legend()
ax3.grid(True, alpha=0.3)

# PPO-specific metrics
ax4 = axes[1, 1]
if agent.training_stats['policy_loss']:
    ax4.plot(agent.training_stats['policy_loss'], label='Policy Loss', alpha=0.7)
    ax4.plot(agent.training_stats['value_loss'], label='Value Loss', alpha=0.7)
    ax4.set_xlabel('Update Step')
    ax4.set_ylabel('Loss')
    ax4.set_title('PPO Training Losses')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 7: Evaluation and Backtesting

### 7.1 Evaluate on Test Data

Now let's evaluate our trained PPO agent on unseen test data.

In [ ]:
def evaluate_agent(env, agent, render=False):
    """
    Evaluate trained agent on environment
    
    Args:
        env: Trading environment
        agent: Trained PPO agent
        render: Whether to print step-by-step actions
    
    Returns:
        metrics: Performance metrics dict
        history: Trading history
    """
    state = env.reset()
    done = False
    total_reward = 0
    
    # Set to evaluation mode
    agent.policy.eval()
    
    with torch.no_grad():
        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            action_probs, _ = agent.policy(state_tensor)
            
            # Use greedy action selection for evaluation
            action = torch.argmax(action_probs).item()
            
            next_state, reward, done, info = env.step(action)
            total_reward += reward
            state = next_state
            
            if render:
                action_names = ['Hold', 'Buy', 'Sell']
                print(f"Action: {action_names[action]}, Portfolio: ${info['portfolio_value']:,.2f}")
    
    # Set back to training mode
    agent.policy.train()
    
    metrics = env.get_metrics()
    metrics['total_reward'] = total_reward
    
    return metrics, env.history

print("✅ Evaluation function defined")

In [ ]:
# Create test environment
test_env = TradingEnvironment(test_df, initial_balance=100000)

# Evaluate on test data
print("📊 Evaluating PPO Agent on Test Data...\n")

test_metrics, test_history = evaluate_agent(test_env, agent)

print("="*60)
print("PPO TRADING AGENT - TEST PERFORMANCE")
print("="*60)
print(f"\n💰 Portfolio Performance:")
print(f"   Initial Balance:    ${100000:>12,.2f}")
print(f"   Final Value:        ${test_metrics['final_value']:>12,.2f}")
print(f"   Total Return:       {test_metrics['total_return']*100:>12.2f}%")
print(f"   Annualized Return:  {test_metrics['annualized_return']*100:>12.2f}%")
print(f"\n📈 Risk Metrics:")
print(f"   Sharpe Ratio:       {test_metrics['sharpe_ratio']:>12.3f}")
print(f"   Max Drawdown:       {test_metrics['max_drawdown']*100:>12.2f}%")
print(f"   Win Rate:           {test_metrics['win_rate']*100:>12.2f}%")
print(f"\n📊 Trading Activity:")
print(f"   Total Trades:       {test_metrics['total_trades']:>12d}")
print("="*60)

In [ ]:
# Compare with Buy & Hold benchmark
def calculate_buy_hold(df, initial_balance=100000):
    """
    Calculate Buy & Hold benchmark returns
    """
    prices = df['Close'].values
    shares = initial_balance / prices[0]
    portfolio_values = shares * prices
    
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    total_return = (portfolio_values[-1] - initial_balance) / initial_balance
    
    # Sharpe ratio
    sharpe = np.sqrt(252) * np.mean(returns) / np.std(returns) if np.std(returns) > 0 else 0
    
    # Max drawdown
    peak = np.maximum.accumulate(portfolio_values)
    drawdown = (peak - portfolio_values) / peak
    max_drawdown = np.max(drawdown)
    
    return {
        'portfolio_values': portfolio_values,
        'total_return': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'final_value': portfolio_values[-1]
    }

# Calculate benchmark
benchmark = calculate_buy_hold(test_df)

print("\n📊 Comparison: PPO Agent vs Buy & Hold")
print("="*60)
print(f"{'Metric':<25} {'PPO Agent':>15} {'Buy & Hold':>15}")
print("-"*60)
print(f"{'Total Return':<25} {test_metrics['total_return']*100:>14.2f}% {benchmark['total_return']*100:>14.2f}%")
print(f"{'Sharpe Ratio':<25} {test_metrics['sharpe_ratio']:>15.3f} {benchmark['sharpe_ratio']:>15.3f}")
print(f"{'Max Drawdown':<25} {test_metrics['max_drawdown']*100:>14.2f}% {benchmark['max_drawdown']*100:>14.2f}%")
print(f"{'Final Value':<25} ${test_metrics['final_value']:>13,.0f} ${benchmark['final_value']:>13,.0f}")
print("="*60)

In [ ]:
# Visualize test performance
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Get aligned dates for test period
test_dates = test_df.index[len(test_df) - len(test_history['portfolio_value']):]

# Portfolio value comparison
ax1 = axes[0]
ax1.plot(test_dates, test_history['portfolio_value'], label='PPO Agent', linewidth=2, color='blue')
benchmark_values = benchmark['portfolio_values'][-len(test_dates):]
ax1.plot(test_dates, benchmark_values, label='Buy & Hold', linewidth=2, color='gray', alpha=0.7)
ax1.fill_between(test_dates, test_history['portfolio_value'], benchmark_values, 
                  where=np.array(test_history['portfolio_value']) > benchmark_values,
                  alpha=0.3, color='green', label='Outperformance')
ax1.fill_between(test_dates, test_history['portfolio_value'], benchmark_values,
                  where=np.array(test_history['portfolio_value']) <= benchmark_values,
                  alpha=0.3, color='red', label='Underperformance')
ax1.set_ylabel('Portfolio Value ($)')
ax1.set_title('PPO Agent vs Buy & Hold - Test Period', fontsize=14)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Position over time
ax2 = axes[1]
positions = test_history['positions'][1:]  # Exclude initial
colors = ['green' if p > 0 else 'gray' for p in positions]
ax2.bar(test_dates[:len(positions)], positions, color=colors, alpha=0.7, width=1)
ax2.set_ylabel('Position (% of Portfolio)')
ax2.set_title('Agent Position Over Time', fontsize=14)
ax2.set_ylim(-0.1, 1.1)
ax2.grid(True, alpha=0.3)

# Actions taken
ax3 = axes[2]
actions = test_history['actions']
action_dates = test_dates[:len(actions)]

# Plot price
price_data = test_df['Close'].values[-len(action_dates):]
ax3.plot(action_dates, price_data, color='black', alpha=0.5, label='SPY Price')

# Mark buy and sell actions
buy_mask = np.array(actions) == 1
sell_mask = np.array(actions) == 2

if np.any(buy_mask):
    buy_dates = action_dates[buy_mask]
    buy_prices = price_data[buy_mask]
    ax3.scatter(buy_dates, buy_prices, color='green', marker='^', s=100, label='Buy', zorder=5)

if np.any(sell_mask):
    sell_dates = action_dates[sell_mask]
    sell_prices = price_data[sell_mask]
    ax3.scatter(sell_dates, sell_prices, color='red', marker='v', s=100, label='Sell', zorder=5)

ax3.set_xlabel('Date')
ax3.set_ylabel('Price ($)')
ax3.set_title('Trading Actions on Price Chart', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Action distribution
action_counts = pd.Series(actions).value_counts().sort_index()
action_names = ['Hold', 'Buy', 'Sell']
print(f"\n📊 Action Distribution:")
for idx, count in action_counts.items():
    pct = count / len(actions) * 100
    print(f"   {action_names[idx]}: {count} ({pct:.1f}%)")

## Part 8: Key Takeaways and Summary

### 8.1 PPO Algorithm Recap

**Core Components:**

1. **Clipped Surrogate Objective**: Prevents destructively large policy updates
   $$L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \hat{A}_t \right) \right]$$

2. **Generalized Advantage Estimation (GAE)**: Reduces variance in advantage estimates
   $$\hat{A}_t = \sum_{l=0}^{\infty} (\gamma \lambda)^l \delta_{t+l}$$

3. **Value Function**: Provides baseline for advantage calculation
   $$L^{VF}(\theta) = (V_\theta(s_t) - V_t^{target})^2$$

4. **Entropy Bonus**: Encourages exploration
   $$L^{ENT}(\theta) = -\mathbb{E}_t[H(\pi_\theta(\cdot|s_t))]$$

### 8.2 Why PPO Works Well for Trading

| Advantage | Explanation |
|-----------|-------------|
| **Stability** | Clipping prevents catastrophic policy changes |
| **Sample Efficiency** | Multiple epochs on same data |
| **Robustness** | Works across different market conditions |
| **Continuous Learning** | Can adapt to changing markets |

### 8.3 Practical Considerations

**Hyperparameter Tuning:**
- `clip_epsilon`: 0.1-0.3 (smaller = more conservative)
- `gamma`: 0.99 (future rewards discount)
- `gae_lambda`: 0.95 (bias-variance tradeoff)
- `n_epochs`: 3-10 (more = better sample efficiency)

**Important Notes:**
- Always include transaction costs
- Test on out-of-sample data
- Consider regime changes in markets
- Monitor for overfitting to training data

In [ ]:
# Final summary
print("="*70)
print("                  WEEK 16 DAY 6: PPO TRADING - SUMMARY")
print("="*70)

print("\n📚 Key Concepts Covered:")
print("   ✓ PPO algorithm fundamentals and motivation")
print("   ✓ Clipped surrogate objective function")
print("   ✓ Generalized Advantage Estimation (GAE)")
print("   ✓ Actor-Critic architecture implementation")
print("   ✓ Full trading environment with technical indicators")
print("   ✓ Training loop with policy updates")
print("   ✓ Evaluation and comparison with benchmarks")

print("\n🔧 Technical Implementation:")
print(f"   • State dimension: {train_env.state_dim} features")
print(f"   • Action space: 3 discrete actions (Hold, Buy, Sell)")
print(f"   • Network: Shared backbone with Actor-Critic heads")
print(f"   • Training episodes: 50")

print("\n📊 Test Performance:")
print(f"   • PPO Return: {test_metrics['total_return']*100:.2f}%")
print(f"   • Benchmark Return: {benchmark['total_return']*100:.2f}%")
print(f"   • PPO Sharpe: {test_metrics['sharpe_ratio']:.3f}")
print(f"   • Benchmark Sharpe: {benchmark['sharpe_ratio']:.3f}")

print("\n💡 Next Steps:")
print("   → Experiment with different reward functions")
print("   → Try continuous action spaces (position sizing)")
print("   → Implement multi-asset trading")
print("   → Add more sophisticated features (order book, sentiment)")
print("   → Explore other RL algorithms (SAC, TD3)")

print("\n" + "="*70)
print("                         🎉 Day 6 Complete!")
print("="*70)